In [1]:
from http import client
from json import load
import pandas
import json
import requests
import untangle
import pymongo
import os
import ssl
# from dotenv import load_dotenv
from bs4 import BeautifulSoup
import time


In [2]:
def get_article_urls(url):
    # Get the RSS feed
    # MSNBC Wayback machine archive urls
    urls = requests.get(url,headers={'User-Agent': 'Custom'}).text
    parse_url = json.loads(urls) #parses the JSON from urls.
    ## Extracts timestamp and original columns from urls and compiles a url list.
    url_list = []
    for i in range(1,len(parse_url)):
        orig_url = parse_url[i][2]
        tstamp = parse_url[i][1]
        waylink = tstamp+'/'+orig_url
        url_list.append(waylink)
    ## Compiles final url pattern.
    final_url_list= []
    for url in url_list:
        final_url = 'https://web.archive.org/web/'+url
        final_url_list.append(final_url)

    link_array = []
    for link in final_url_list:
        import ssl
        if hasattr(ssl, '_create_unverified_context'):
            ssl._create_default_https_context = ssl._create_unverified_context
            
        second_http =  link.index('/http')
        link =  link[:second_http] + 'if_/' + link[second_http+1:]
        
        
        print(link)
        document = untangle.parse(link)
        items = document.rss.channel.item
        for item in items:
            link = item.link.cdata.split('?')[0]
            if link not in link_array:
                link_array.append(link)
    link_array.pop(0)
    return link_array

In [3]:
def get_article_data(article_url):
    #article_html = requests.get(article_url).text
    link = article_url.split("https://")
    article_html = requests.get(f"https://webcache.googleusercontent.com/search?q=cache%3Ahttp%3A//{link[1]}",headers={'User-Agent': 'Custom'}).text

        
    # Parse the HTML
    soup = BeautifulSoup(article_html, 'html.parser')
    #title = soup.find(class_="css-1vkm6nb ehdk2mb0").text
    title = soup.find('title').text.strip('- TASS')
    time = ''
    #time = soup.find('meta',property="article:modified_time")['content']
    
    body_list = []
    for item in soup.find_all('div', class_="text-block"):
        body_list.append(item.text)
    # body_list = body_list[0].split('\n')
        
    article_data_obj = {
        'title': title,
        'posted_at': time,
        'body': ' '.join(body_list)
    }
    
    return article_data_obj
    

In [4]:
tass_rss_url = 'http://web.archive.org/cdx/search/cdx?url=tass.com/rss/v2.xml&from=20220313&to=20220321&output=json'
articles = get_article_urls(tass_rss_url)


https://web.archive.org/web/20220313005828if_/http://tass.com/rss/v2.xml
https://web.archive.org/web/20220315040523if_/http://tass.com/rss/v2.xml
https://web.archive.org/web/20220315085848if_/https://tass.com/rss/v2.xml
https://web.archive.org/web/20220316034046if_/http://tass.com/rss/v2.xml
https://web.archive.org/web/20220316052454if_/https://tass.com/rss/v2.xml
https://web.archive.org/web/20220318035523if_/http://tass.com/rss/v2.xml
https://web.archive.org/web/20220318035751if_/https://tass.com/rss/v2.xml
https://web.archive.org/web/20220319232751if_/http://tass.com/rss/v2.xml
https://web.archive.org/web/20220320064956if_/http://tass.com/rss/v2.xml
https://web.archive.org/web/20220320065135if_/https://tass.com/rss/v2.xml
https://web.archive.org/web/20220321040934if_/http://tass.com/rss/v2.xml
https://web.archive.org/web/20220321080625if_/https://tass.com/rss/v2.xml


In [8]:
articles[41]

'https://tass.com/science/1421121'

In [ ]:
articleList = []
count = 0
for article in articles:
    print(count)
    article_data_obj = get_article_data(article)
    print(article_data_obj['title'])
    articleJson = {
            'title': article_data_obj['title'],
            # 'posted_at': article_data_obj['posted_at'],
            'body': article_data_obj['body'],
        }

    articleList.append(articleJson)
    jsonString = json.dumps(articleList)
    jsonFile = open("tass.json", "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    count += 1
df = pandas.read_json (r'tass.json')
df.to_csv(r'tass.csv', index = None)

In [39]:
articleList = []
count = 0

In [58]:
for article in range(count, len(articles)):
    print(count)
    article_data_obj = get_article_data(articles[article])
    print(article_data_obj['title'])
    if "webcache.googleusercontent.com" in article_data_obj['title']:
        break
    articleJson = {
            'title': article_data_obj['title'],
            # 'posted_at': article_data_obj['posted_at'],
            'body': article_data_obj['body'],
        }

    articleList.append(articleJson)
    count += 1

477
mmonia leak occurs at chemical plant in Ukraine’s Sumy — local authorities -  Emergencies
478
Over 10 people hospitalized in Rostov-on-Don with burns after heating pipeline failure -  Emergencies


In [55]:
count = 424

In [59]:
jsonString = json.dumps(articleList)
jsonFile = open("tass.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [60]:
df = pandas.read_json (r'tass.json')
df.to_csv(r'tass.csv', index = None)